# ITMAL Demo: Covariance Matrix


NOTE: some code snippets takenfrom 

> https://datascienceplus.com/understanding-the-covariance-matrix/

In [47]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from libitmal import utils as itmalutils

def PrintNpInfo(X, varname):
    #if not isinstance(X, np.array):
    #    print("ftype({varname}), len(X)={len(X)} (WARN: not a numpy array)")
    #else:
    print(f"type({varname})={type(X)}, {varname}.ndim={X.ndim}, {varname}.shape={X.shape}")

def Save_fig(fig_id, tight_layout=True, fig_extension='png', resolution=300):
    import os
    path = os.path.join("/home/cef/", fig_id + "." + fig_extension) 
    print(f"Saving figure to file '{path}'")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
def GenerateData(N=100):
    # Normal distributed x and y vector with mean 0 and standard deviation 1
    x = np.random.normal(0, 2, N)
    y = np.random.normal(0, 1, N)
    X = np.vstack((x, y)).T
    return X

def ScaleAndRotate(X, r=0.77, sx=0.7, sy=3.4):
    # Scaling matrix
    Scale = np.array([[sx, 0], [0, sy]])

    # Rotation matrix
    theta = r*np.pi
    c, s = np.cos(theta), np.sin(theta)
    Rot = np.array([[c, -s], [s, c]])

    # Transformation matrix
    T = Scale.dot(Rot)

    # Apply transformation matrix to X
    Y = X.dot(T)
    return Y

def DoPlot(X):
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = (4, 4)
    plt.scatter(X[:, 0], X[:, 1])
    plt.title('Generated Data')
    plt.axis('equal');
    plt.xlabel('feature $\lambda_1$')
    plt.ylabel('feature $\lambda_2$')
    #Save_fig('covariance') 

# Covariance
def Cov(x, z, bias=True):
    assert len(x)==len(z) 
    n = len(x) - (0 if bias else 1)
    assert n>0
    xbar, zbar = x.sum()/n, z.sum()/n
    xbar, zbar = x.mean(), z.mean()
    #return np.sum((x - xbar)*(z - zbar))/n # several diff. methods here
    return np.sum(x*z - xbar*zbar)/n

# Covariance matrix
def Auto_Cov_Mat(X, bias=True):
    return np.array([\
        [Cov(X[0], X[0], bias), Cov(X[0], X[1], bias)], \
        [Cov(X[1], X[0], bias), Cov(X[1], X[1], bias)]  \
    ])

X = GenerateData()
X = ScaleAndRotate(X, 0.9, 1, 10)

print(f'The shapes...X.shape={X.shape}, (X.T)[0].shape={(X.T)[0].shape}')
#def PrintMatrix(X, label="", precision=2, threshold=100, edgeitems=1, linewidth=80, suppress=True):
itmalutils.PrintMatrix(X, label="X=")

# Calculate covariance matrix, the .T should be removed!
C0_biased  =Auto_Cov_Mat(X.T, bias=True)
C1_biased  =np.cov      (X  , bias=True, rowvar=False)
C0_unbiased=Auto_Cov_Mat(X.T, bias=False)
C1_unbiased=np.cov      (X  , bias=False,rowvar=False)

print()
itmalutils.PrintMatrix(C0_biased,           label="C0_biased  =")
itmalutils.PrintMatrix(C1_biased,           label="C1_biased  =")
itmalutils.PrintMatrix(C0_biased-C1_biased, label="diff       =")

print()
itmalutils.PrintMatrix(C0_unbiased,            label="C0_unbiased=")
itmalutils.PrintMatrix(C0_unbiased,            label="C0_unbiased=")
itmalutils.PrintMatrix(C0_unbiased-C1_unbiased,label="diff       =")

print()
itmalutils.AssertInRange(C0_biased,  C1_biased)
itmalutils.AssertInRange(C0_unbiased,C1_unbiased)


The shapes...X.shape=(100, 2), (X.T)[0].shape=(100,)
X=[[  4.47 -14.11]
   ...
   [  0.22   4.71]]

C0_biased  =[[ 14.35 -32.8 ]
             [-32.8  102.19]]
C1_biased  =[[ 14.35 -32.8 ]
             [-32.8  102.19]]
diff       =[[ 0.  0.]
             [ 0. -0.]]

C0_unbiased=[[ 14.49 -33.13]
             [-33.13 103.22]]
C0_unbiased=[[ 14.49 -33.13]
             [-33.13 103.22]]
diff       =[[ 0.  0.]
             [ 0. -0.]]



In [52]:
# Cross Covariance, nearly full matrix implementation
def Cross_Cov_Mat(X, Z, bias=True, debug=False):
    assert isinstance(X, np.ndarray)
    assert isinstance(Z ,np.ndarray)
    assert X.ndim==2 and Z.ndim==2
    assert X.shape==Z.shape 
   
    N, M = X.shape
    n = N if bias else N-1
    assert N>0 and n>0
    
    Xbar = X.mean(axis=0)[:, np.newaxis]
    Zbar = Z.mean(axis=0)[:, np.newaxis]

    if debug:
        PrintNpInfo(Xbar, "Xbar")
        PrintNpInfo(Xbar, "Zbar")
    
    C = np.zeros((M, M))
    for i in range(N):
        xi = X[i, :, np.newaxis] # needs new axis to do row/column dot product
        zi = Z[i, :, np.newaxis]
        c = (np.dot(xi, zi.T) - np.dot(Xbar, Zbar.T))/n
        assert c.shape==C.shape
        C += c
    
    if debug:
        PrintNpInfo(C, "C")
        
    return C

# TEST it..
i=2
while i<2000:
    i *= 2
    X = ScaleAndRotate(GenerateData(i), 0.9, 1, 10)
    Z = ScaleAndRotate(GenerateData(i), 0.8, 1, 12)
    
    print(f"TEST[{i}]..")
    #PrintNpInfo(X, "  X=")
    #PrintNpInfo(Z, "  Z=")

    # Calculate covariance matrix
    C1_biased  =np.cov      (X, Z, bias=True, rowvar=False) # Z var have no effect??
    C1_unbiased=np.cov      (X, Z, bias=False,rowvar=False)
    
    C2_biased  =Cross_Cov_Mat(X, X, bias=True) # notice wrong X, X param. due to prob. with np.cov
    C2_unbiased=Cross_Cov_Mat(X, X, bias=False)

    itmalutils.PrintMatrix(C2_biased,   label=f"  C2_biased  =")
    itmalutils.PrintMatrix(C2_unbiased, label=f"  C2_unbiased=")

    itmalutils.AssertInRange(C2_biased,  C1_biased)
    itmalutils.AssertInRange(C2_unbiased,C1_unbiased)

    print("  OK\n")

TEST[4]..
  C2_biased  =[[  4.67 -15.41]
               [-15.41  53.53]]
  C2_unbiased=[[  6.23 -20.54]
               [-20.54  71.37]]
  OK

TEST[8]..
  C2_biased  =[[ 17.08 -22.  ]
               [-22.    70.44]]
  C2_unbiased=[[ 19.52 -25.15]
               [-25.15  80.51]]
  OK

TEST[16]..
  C2_biased  =[[  9.14 -19.83]
               [-19.83  76.29]]
  C2_unbiased=[[  9.75 -21.16]
               [-21.16  81.37]]
  OK

TEST[32]..
  C2_biased  =[[ 22.04 -39.06]
               [-39.06 114.08]]
  C2_unbiased=[[ 22.75 -40.32]
               [-40.32 117.76]]
  OK

TEST[64]..
  C2_biased  =[[ 18.81 -43.32]
               [-43.32 133.1 ]]
  C2_unbiased=[[ 19.11 -44.01]
               [-44.01 135.21]]
  OK

TEST[128]..
  C2_biased  =[[ 12.87 -28.27]
               [-28.27  91.17]]
  C2_unbiased=[[ 12.97 -28.49]
               [-28.49  91.89]]
  OK

TEST[256]..
  C2_biased  =[[ 12.35 -27.51]
               [-27.51  88.15]]
  C2_unbiased=[[ 12.39 -27.62]
               [-27.62  88.49]]
  OK


REVISIONS| |
:- | :- |
2018-02-25| CEF, initial.
2018-02-14| CEF, added refs.
2023-02-19| CEF, add full cross covariance calc and test functionality, still missing fix of .T in Auto_Cov_Mat().